Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
import geopy.distance
import networkx as nx
import functools

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

# Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

### Collection of cities, divided by country

In [2]:
countries=[
    

    {
        'name': 'Vanuatu',
        'cities': pd.read_csv('cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon']),
        'dist_matrix':[]
    },

    {
        'name':'Italy',
        'cities':pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon']),
        'dist_matrix':[]
    },

    {
        'name':'Russia',
        'cities': pd.read_csv('cities/russia.csv', header=None, names=['name', 'lat', 'lon']),
        'dist_matrix':[]
    },

    
    {
        'name':'USA',
        'cities':pd.read_csv('cities/us.csv', header=None, names=['name', 'lat', 'lon']),
        'dist_matrix':[]
    },
    
    {
        'name':'China',
        'cities': pd.read_csv('cities/china.csv', header=None, names=['name', 'lat', 'lon']),
        'dist_matrix':[]
    },
]




### Distance matrix calculation

In [3]:
for country in countries:
    cities=country['cities']
    dist_matrix = np.zeros((len(cities), len(cities)))
    for c1, c2 in combinations(cities.itertuples(), 2):
        dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = geopy.distance.geodesic(
            (c1.lat, c1.lon), (c2.lat, c2.lon)
        ).km
    country['dist_matrix']=dist_matrix

## TSP solutions
Countries indexes:
+ 0: Vanuatu
+ 1: Italy
+ 2: Russia
+ 3: USA
+ 4: China

### Greedy algorithm
Implementing a fast greedy algorithm

#### Utility functions

In [4]:
def tsp_validation(cities,sol):

    '''
    Checks the validity of the given solution.
    The tsp problem is solved if:
    1) The solution covers all the cities
    2) The solution is a cycle
    '''

    #tsp mut be a cycle
    assert sol[0]==sol[-1]

    #the tsp must cover all the cities
    assert set(sol)==set(range(len(cities)))

def cost(sol,dist_matrix):
    '''
    Return the cost of the given tsp solution. given the distance matrix
    '''

    cost = 0
    for c1, c2 in zip(sol, sol[1:]):
        cost += dist_matrix[c1, c2]
    return cost

#### Greedy TSP definition

In [30]:
def tsp_greedy(cities,distance_matrix):

    visited = np.full(len(cities),False)
    start = 0
    current_city = start
    dist=np.copy(distance_matrix)
    visited[start]=True
    voyage=[]
    voyage.append(start)
    journey_diary=''
    steps = 0

    while not np.all(visited):

        dist[: ,current_city]=np.inf
        distance_vector = dist[current_city]
        next_stop=np.argmin(distance_vector)
        voyage.append(next_stop)
        
        journey_diary += f'Start: {cities.at[current_city,'name']} --> Arrival: {cities.at[next_stop,'name']} || Kilometers travelled: {distance_vector[next_stop]:.2f}\n'
        steps+=1
        current_city=next_stop
        visited[current_city]=True
    
    voyage.append(0)
    journey_diary += f'Finally going home!\t Start: {cities.at[voyage[-2],'name']} --> Arrival: {cities.at[0,'name']} || Kilometers travelled: {distance_matrix[voyage[-2],0]:.2f}\n'
    journey_diary += f'Total kilometers travelled: {cost(voyage,distance_matrix):.2f}\n'
    journey_diary += f'Total numer of "steps": {steps}'

    return (voyage,journey_diary)



#### Solving the TSP
The solutions are stored in a txt file

In [31]:
for country in countries:
    print(f'Now doing: {country['name']}')
    
    (greedy_voyage,greedy_diary)=tsp_greedy(country['cities'],country['dist_matrix'])
    with open(f'{country['name']} greedy voyage journal.txt','w',encoding='UTF-8') as f:
        f.write(greedy_diary)


Now doing: Vanuatu
(8, 8)
Now doing: Italy
(46, 46)
Now doing: Russia
(167, 167)
Now doing: USA
(326, 326)
Now doing: China
(726, 726)


### Evolutionary algorithms